In [1]:
import sleap
import numpy as np
import pandas as pd
import os
from pathlib import Path

sleap.versions()

SLEAP: 1.3.3
TensorFlow: 2.7.0
Numpy: 1.21.6
Python: 3.7.12
OS: Windows-10-10.0.22621-SP0


In [2]:
# scale
px_per_mm = 10.6

In [3]:
# Creates `new_directory` where model evluation data will be saved
new_directory = "canola_primary_model_eval"
os.makedirs(f"{new_directory}", exist_ok=True)

In [4]:
# canola primary root models
models = r"""
plantwise_test_splits/plantwise_split_canola_pr.v000/models/230317_121725.multi_instance.n=305
plantwise_test_splits/plantwise_split_canola_pr.v001/models/230317_123935.multi_instance.n=302
plantwise_test_splits/plantwise_split_canola_pr.v002/models/230317_130113.multi_instance.n=303
""".strip().split()

In [5]:
# all px converted to mm
dist_p50, dist_p90, dist_p95, dist_p99, dist_avg, dist_std = [], [], [], [], [], []
dists = []
vis_prec, vis_recall = [], []
oks_map, oks_mar = [], []
model_names, root_names = [], []

for model in models:
    metrics = sleap.load_metrics(model, split="test")
    root_name = Path(model).parts[1]
    model_name = Path(model).parts[3]
    
    print(f"{root_name}")
    print(f"{model_name}")
    print("Error distance (50%):", metrics["dist.p50"] / px_per_mm)
    print("Error distance (90%):", metrics["dist.p90"] / px_per_mm)
    print("Error distance (95%):", metrics["dist.p95"] / px_per_mm)
    print("Error distance (99%):", metrics["dist.p99"] / px_per_mm)
    print("Error distance (avg):", metrics["dist.avg"] / px_per_mm)
    print("Error std:", np.nanstd(metrics["dist.dists"].flatten()) / px_per_mm)
    print("Vis Precision:", metrics["vis.precision"])
    print("Vis Recall:", metrics["vis.recall"])
    print("oks mAP:", metrics["oks_voc.mAP"])
    print("oks mAR:", metrics["oks_voc.mAR"])
    
    dist_p50.append(metrics["dist.p50"] / px_per_mm)
    dist_p90.append(metrics["dist.p90"] / px_per_mm)
    dist_p95.append(metrics["dist.p95"] / px_per_mm)
    dist_p99.append(metrics["dist.p99"] / px_per_mm)
    dist_avg.append(metrics["dist.avg"] / px_per_mm)
    vis_prec.append(metrics["vis.precision"])
    vis_recall.append(metrics["vis.recall"])
    oks_map.append(metrics["oks_voc.mAP"])
    oks_mar.append(metrics["oks_voc.mAR"])
    model_names.append(model_name)
    root_names.append(root_name)
    dist_std.append(np.nanstd(metrics["dist.dists"].flatten()) / px_per_mm)
    dists.append(metrics["dist.dists"].flatten() / px_per_mm)

FileNotFoundError: [Errno 2] No such file or directory: 'plantwise_test_splits/plantwise_split_canola_pr.v000/models/230317_121725.multi_instance.n=305'

In [ ]:
# Make a dataframe with summary statistics of model evaluation
df_summary = pd.DataFrame({
    "Root": root_names,
    "Model": model_names,
    "Error distance (50%) mm": dist_p50,
    "Error distance (90%) mm": dist_p90,
    "Error distance (95%) mm": dist_p95,
    "Error distance (99%) mm": dist_p99,
    "Error distance (avg) mm": dist_avg,
    "Error std mm": dist_std,
    "Vis Precision": vis_prec,
    "Vis Recall": vis_recall,
    "oks mAP": oks_map,
    "oks mAR": oks_mar,
})
df_summary

In [ ]:
# Save summary of error metrics per model
df_summary.to_csv(f"{new_directory}/error_metrics_summary.csv", index=False)

In [ ]:
# Make a dataframe with the all three test sets localization error distributions
df_error = pd.concat([
    pd.DataFrame({
        "model": [f"{model_names[0]}"] * len(dists[0]),
        "test_dists": list(dists[0]),

    }),
    pd.DataFrame({
        "model": [f"{model_names[1]}"] * len(dists[1]),
        "test_dists": list(dists[1]),

    }),
    pd.DataFrame({
    "model": [f"{model_names[2]}"] * len(dists[2]),
    "test_dists": list(dists[2]),

    }),
])
df_error

In [ ]:
# Save error distributions 
df_error.to_csv(f"{new_directory}/error_dists.csv", index=False)